In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# Create a synthetic temperature dataset (can be replaced with real data)
np.random.seed(42)
temp_data = np.cumsum(np.random.randn(1000) * 0.5 + 0.1) + 20  # Random walk with trend


In [ ]:
# Convert to supervised dataset
def create_dataset(data, time_steps=10):
    X, y = [], []
    for i in range(len(data) - time_steps):
        X.append(data[i:i+time_steps])
        y.append(data[i+time_steps])
    return np.array(X), np.array(y)

# Normalize data
scaler = MinMaxScaler()
temp_data_scaled = scaler.fit_transform(temp_data.reshape(-1, 1))

# Create dataset
time_steps = 10
X, y = create_dataset(temp_data_scaled, time_steps)
X = X.reshape((X.shape[0], X.shape[1], 1))  # 3D input for LSTM

In [ ]:
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

In [ ]:
model = Sequential([
    LSTM(50, activation='relu', return_sequences=True, input_shape=(time_steps, 1)),
    LSTM(50, activation='relu'),
    Dense(1)
])

model.compile(optimizer='adam', loss='mse')

# Train the model
history = model.fit(X_train, y_train, epochs=20, batch_size=16,
                    validation_data=(X_test, y_test), shuffle=False)

In [ ]:
predicted_temp = model.predict(X_test)
predicted_temp = scaler.inverse_transform(predicted_temp)
y_test_actual = scaler.inverse_transform(y_test.reshape(-1, 1))

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(y_test_actual, label='Actual Temperature', color='blue')
plt.plot(predicted_temp, label='Predicted Temperature', color='red')
plt.xlabel('Time')
plt.ylabel('Temperature')
plt.legend()
plt.title('Actual vs Predicted Temperature')
plt.show()

In [ ]:
plt.figure(figsize=(8, 5))
plt.plot(history.history['loss'], label='Train Loss', color='blue')
plt.plot(history.history['val_loss'], label='Validation Loss', color='red')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Loss Curve')
plt.show()